# GVCF Alignment Quality

This notebook calculates and plots features relevant to alignment quality - such as number of SNPs, number of indels, and alignment coverage - for a set of gvcf files. These gvcf files are assumed to have been produced by aligning whole assemblies and/or WGS data to a reference genome.

## Requirements

* A directory containing one or more gvcf files, which may be gzipped
* Docker
* Python packages: matplotlib, numpy, pandas, itables, seaborn

## Configuration and User Variables

Run the following cells to set up your environment. In the cell labeled "Edit Me", change the directories to match your filesystem. Notice that Docker-specific filepaths may differ from the absolute filepath on your machine.

Later in this notebook, you will see other cells labeled "Edit Me". Take a moment to look over the variables in these cells and change them to suit your analysis. 

It is recommended that you run all cells in order. Some cells depend on the results from previous cells, and they will throw errors if you skip them. 

In [ ]:
# import required libraries

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas
from collections import Counter
import math
from itables import init_notebook_mode
import seaborn as sns

tmp = init_notebook_mode(all_interactive=True)

In [ ]:
###########
# EDIT ME #
###########

# working directory
working_dir = "/path/to/working/directory/"

# Note: for the following variables, please use the path relative to the working directory
# The reason for this is that the absolute path changes within the docker

# the directory containing the gvcf files
gvcf_dir = "/gvcf/"

# the next two output files can also be nested within a subdirectory
# make sure that directory exists before continuing

# the main output file - a table of summary statistics
summary_file_name = "metrics/sorghum_gvcf_metrics.tsv"

# optional output file - lists the sizes of indels
# if you don't want this output, use indels_file_name = ""
indels_file_name = "metrics/sorghum_gvcf_indels.tsv"

# path to figure directory, relative to working_dir
fig_dir = "/figures/"

# Docker-specific parameters

# Docker image and version
DOCKER_VERSION="maizegenetics/phg:latest"
# Docker path - location where Docker is installed
# Note: on CBSU machines this is docker1
DOCKER_PATH="docker"

# Do not change these; they are docker-specific paths
PHG_VCF_DIR="/phg/" + gvcf_dir
PHG_OUT_FILE="/phg/" + summary_file_name

if indels_file_name == "":
    PHG_INDEL_FLAG = ""
else:
    PHG_INDEL_FLAG = "-indelFile /phg/" + indels_file_name 

In [ ]:
# make figure folder if it doesn't exist

if not os.path.exists(working_dir + "/" + fig_dir):
    os.mkdir(working_dir + "/" + fig_dir)

In [ ]:
# Run VCFMetricsPlugin

! {DOCKER_PATH} run --name vcf_metrics --rm \
    -v {working_dir}/:/phg/ \
    -t {DOCKER_VERSION} \
    /tassel-5-standalone/run_pipeline.pl -Xmx50G -debug \
    -VCFMetricsPlugin \
        -vcfDir {PHG_VCF_DIR} \
        -outFile {PHG_OUT_FILE} \
        {PHG_INDEL_FLAG} \
    -endPlugin

# Summary Table

The cell below displays the summary table that the plugin writes. Summaries for each assembly are on rows that begin with \[assembly name\]\_ALL. Following the overall summary are rows containing overviews of each chromosome represented in the gvcf. This notebook shows only the whole-assembly summary rows. 

You may wish to add more columns to this table. For example, a "shortNames" column could contain abbreviated assembly names for cleaner labeling on figures. Or, a "group" column might contain metadata about the technology used to produce the assemblies, so that points on a graph may be grouped and/or color-coded. The table can be opened and edited in Excel or any simple text editing program. If using a basic text editor, make sure to separate columns by tabs and not spaces. If you add columns to the table, please re-run the cell below so that the new columns appear.

In [ ]:
# load in metrics table, and save summary lines to summary_table
metrics_table = pandas.read_csv(working_dir + "/" + summary_file_name, sep="\t", index_col=False)

summary_table = metrics_table[[name.endswith("_ALL") for name in metrics_table["name"]]]
summary_table

In [ ]:
###########
# EDIT ME #
###########

# Display these assembly names on figures. Default is sample_names = summary_table["name"]

# To change use a column in the summary table (you may directly edit the .tsv file to add columns)
# Select the appropriate column with the format: sample_names = summary_table["column_name"]
sample_names = summary_table["name"]

# If you wish to color-code data points based on a categorical variable you supply (e.g. alignment software), 
# specify the groups here. Like sample_names, add a column to the .tsv file
# If you do not wish to color-code, set sample_groups = list()

sample_groups = list()


# Do not edit this. Having the sample names/groups as lists makes some things easier later on
sample_names = list(sample_names)
sample_groups = list(sample_groups)

# Percent Alignment and Mapping

The following cell shows the percentage of bases aligned/mapped relative to the reference for each gvcf in the selected directory. In this context, "aligned" and "mapped" have separated definitions. A base is considered to be aligned if the base in the reference corresponds to a single base in the query. The bases do not need to be identical - SNPs count as well as reference blocks. In the exaple alignment below, there are 10 aligned bases - 7 that are the same as the reference, and 3 SNPs.

``ref: GATC--GGATCAAAC``  
``     | ||  ||   |  |``  
``qry: GGTCTCGG---ATGC``

Every base that is aligned is also mapped under this notebook's definition, but mapped bases also include insertions and deletions of any size. The only bases that are considered "unmapped" are those that are not included in any gvcf record, whether as part of a reference block or a variant. We have usually seen unmapped bases around the beginnings and ends of contigs. 

All percentages below are relative to the reference genome, i.e. the percentage of bases in the reference genome that are aligned or mapped.

In [ ]:
# plot percent of base pairs aligned/mapped

percents = summary_table["percentRefAligned"].tolist()
percents.extend(summary_table["percentRefMapped"].tolist())

data = pandas.DataFrame.from_dict({"name": sample_names + sample_names,
        "percent": percents, 
        "count": ["aligned"] * len(sample_names) + ["mapped"] * len(sample_names)})

if not len(sample_groups) == 0:
    data["group"] = sample_groups + sample_groups
    data.sort_values(by=["group", "percent"], inplace=True, ascending=[True, True])
else:
    data.sort_values(by="percent", inplace=True, ascending=False)


fig, ax = plt.subplots()

if len(sample_groups) == 0:
    sns.scatterplot(data=data, x="name", y="percent", ax=ax, style="count", markers=["o", "s"])
else:
    sns.scatterplot(data=data, x="name", y="percent", hue="group", ax=ax, style="count", markers=["o", "s"])

#Format axis
ax.set_ylim([0, 1])
ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
ax.set_yticklabels(["0%", "20%", "40%", "60%", "80%", "100%"])
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=-90, ha="right")
ax.set(xlabel=None, ylabel=None)
ax.set_axisbelow(True)

plt.grid()

plt.title("Percentage of bases aligned and mapped to the reference genome")

# Can change this parameter if bottom test gets cut off
plt.gcf().subplots_adjust(bottom=0.25)
plt.legend(bbox_to_anchor=(1.0, 1.0))

fig.savefig(working_dir + "/" + fig_dir + "/" + "percent_bases_aligned.svg")

plt.show()

# Number of SNPs

The following cell generates a figure showing the number of SNPs present in each GVCF file. SNPs may include real SNPs as well as sequencing errors and missing bases. Indels and unmapped regions are not included.

In [ ]:
# plot number of SNPs

data = pandas.DataFrame.from_dict({
    "name": sample_names,
    "snps": summary_table["numSNPs"]
})

if len(sample_groups) == 0:
    data.sort_values(by=["snps"], inplace=True, ascending=[True])
else:
    data["group"] = sample_groups
    data.sort_values(by=["group", "snps"], inplace=True, ascending=[True, True])

fig, ax = plt.subplots()

if len(sample_groups) == 0:
    sns.barplot(data=data, x="name", y="snps", dodge=False)
else:
    sns.barplot(data=data, x="name", y="snps", hue="group", dodge=False)

ax.set_xticklabels(ax.get_xticklabels(), rotation=-90, ha="right")

ax.set(xlabel="count", ylabel=None)
ax.set_axisbelow(True)
ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.grid()

plt.title("Number of SNPs relative to reference")
plt.gcf().subplots_adjust(bottom=0.25, left=0.15)
plt.legend(bbox_to_anchor=(1.0, 1.0))
fig.savefig(working_dir + "/" + fig_dir + "/" + "num_snps.svg")
plt.show()

# Number of Indels

The following cell generates a figure showing the number of indels in each gvcf. Indels are defined here as variants where the length of the reference and the length of the alternate are different. They do not include unmapped regions. 
In this graph, the number of insertions and deletions are separated and shown as a stacked bar (deletions in lighter color on top, insertions in darker color on bottom).

In [ ]:
data = pandas.DataFrame({
    "total": summary_table["numIndels"].tolist()
}, index=sample_names)

insertions = summary_table["numIns"].tolist()
deletions = summary_table["numDel"].tolist()

if len(sample_groups) == 0:
    data["Ins"] = insertions
    data["Del"] = deletions
    
    colors = iter([plt.cm.tab20(i) for i in range(2)])
    cmap = [next(colors) for x in range(2)]
    
    data.sort_values(by=["total"], inplace=True, ascending=[True])
    data.drop(["total"], inplace=True, axis=1)
else:    
    
    data["group"] = sample_groups
    tmp_counter = 0
    for group in np.unique(sample_groups):
        tmp_counter += 1
        subgroup = group + " - Ins"
        data[subgroup] = [insertions[x] if sample_groups[x] == group else 0 for x in range(len(sample_names))]
        subgroup = group + " - Del"
        data[subgroup] = [deletions[x] if sample_groups[x] == group else 0 for x in range(len(sample_names))]

    
    colors = iter([plt.cm.tab20(i) for i in range(tmp_counter * 2)])
    cmap = [next(colors) for x in range(tmp_counter * 2)]

    data.sort_values(by=["group", "total"], inplace=True, ascending=[True, True])
    data.drop(["group", "total"], inplace=True, axis=1)

    
data.plot(kind="bar", stacked=True, color=cmap, zorder=2.5)

ax = plt.gca()


ax.set(xlabel="count", ylabel=None)

ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.set_xticklabels(ax.get_xticklabels(), rotation=-90, ha="right")

plt.grid()

plt.title("Number of indels relative to reference")
plt.legend(bbox_to_anchor=(1.0, 1.0))

plt.gcf().subplots_adjust(bottom=0.25, left=0.15, right=1.0)

plt.gcf().savefig(working_dir + "/" + fig_dir + "/" + "num_indels.svg")

plt.show()

# Largest Indels

The next cell generates a figure showing the largest indel in each gvcf file. The shape of the point indicates whether that indel is an insertion or a deletion. 

In [ ]:
ins = summary_table["largestInsSize"].tolist()
delet = summary_table["largestDelSize"].tolist()

largest = summary_table["largestIndel"].tolist()

indels = ["Insertion" if ins[x] > delet[x] else "Deletion" for x in range(len(sample_names))]

data = pandas.DataFrame.from_dict({
    "name": sample_names,
    "type": indels,
    "sizes": largest
})

if len(sample_groups) == 0:
    data.sort_values(by=["sizes"], inplace=True, ascending=[True])
else:
    data["group"] = sample_groups
    data.sort_values(by=["group", "sizes"], inplace=True, ascending=[True, True])

fig, ax = plt.subplots()

if len(sample_groups) == 0:
    sns.scatterplot(data=data, x="name", y="sizes", ax=ax, style="type", markers=["X", "o"])
else:
    sns.scatterplot(data=data, x="name", y="sizes", hue="group", ax=ax, style="type", markers=["X", "o"])

ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=-90, ha="right")
ax.set(xlabel=None, ylabel="size")
ax.set_axisbelow(True)

plt.grid()
plt.title("Largest indel sizes")
plt.gcf().subplots_adjust(bottom=0.25, left=0.15)
plt.legend(bbox_to_anchor=(1.0, 1.0))

fig.savefig(working_dir + "/" + fig_dir + "/" + "largest_indels.svg")
plt.show()